In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

import tensorflow as tf
tf.test.gpu_device_name()


Mounted at /content/drive


'/device:GPU:0'

In [ ]:
Data = pd.read_csv("/content/drive/MyDrive/DeepLearning - Headlines Proj/PreprocessedData0506.csv")
Data['preprocessed_headline_text'] = Data['preprocessed_headline_text'].fillna('<missing>').astype(str)
Data

,publish_date,headline_category,headline_text,primary_category,year,preprocessed_headline_text
0,1996-01-02,news,uup sees possibility of voting major out,news,1996,uup see possibl vote major
1,1996-01-02,news,pubs targeted as curbs on smoking are extended,news,1996,pub target curb smoke extend
2,1996-01-02,news,papers reveal secret links with o'neill cabinet,news,1996,paper reveal secret link o'neil cabinet
3,1996-01-02,news,domestic chaos as italy takes eu presidency,news,1996,domest chao itali take eu presid
4,1996-01-02,news,learning about the star to which we owe life,news,1996,learn star owe life
...,...,...,...,...,...,...
1520352,2021-06-30,news.politics.oireachtas,reserve members of defence forces to be allowe...,news,2021,reserv member defenc forc allow serv oversea f...
1520353,2021-06-30,culture,maureen dowd: joe biden is 'crazy about the ir...,culture,2021,maureen dowd joe biden 'crazi irish plan celeb...
1520354,2021-06-30,sport.others,andy murray rolls back the years to own centre...,sport,2021,andi murray roll back year centr court
1520355,2021-06-30,news.health,delta variant could do 'significant damage' in...,news,2021,delta variant could 'signific damag partial va...


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset
import datetime
import numpy as np
import random
import torch.nn as nn

## Testing each of candidate checkpoints' preformances 

In [ ]:
from sklearn.preprocessing import LabelEncoder

checkpoint_160K_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-160000"
checkpoint_200K_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-200000"
checkpoint_220K_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-220000"
checkpoint_280K_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-280000"

# Load the models
models = {
    "checkpoint_160000": DistilBertForSequenceClassification.from_pretrained(checkpoint_160K_path),
    "checkpoint_200000": DistilBertForSequenceClassification.from_pretrained(checkpoint_200K_path),
    "checkpoint_220000": DistilBertForSequenceClassification.from_pretrained(checkpoint_220K_path),
    "checkpoint_280000": DistilBertForSequenceClassification.from_pretrained(checkpoint_280K_path)
}

# Assume your Data variable is a pandas DataFrame
preprocessed_headlines = Data['preprocessed_headline_text'].tolist()

# Encode the labels with the same encoder you used during training
#encoder = LabelEncoder()
label_dict = {"business": 0, "culture": 1, "lifestyle": 2, "news": 3, "opinion": 4, "sport": 5}

# Transform the labels
labels = Data['primary_category'].apply(lambda x: label_dict[x]).values.tolist()


# Tokenize the preprocessed_headlines
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
encoded_data = tokenizer.batch_encode_plus(preprocessed_headlines, truncation=True, padding=True, max_length=15)




In [ ]:
input_ids = torch.tensor(encoded_data["input_ids"])
attention_masks = torch.tensor(encoded_data["attention_mask"])

#original_dataset = TensorDataset(input_ids, attention_masks, torch.tensor(labels))
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Construct a CustomDataset
original_dataset = CustomDataset(encoded_data, labels)


# Compute metrics function for validation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

results = {}
for model_name, model in models.items():
    trainer = Trainer(
        model=model,                         
        compute_metrics=compute_metrics,     
    )

    result = trainer.evaluate(original_dataset)
    results[model_name] = result
    print (result)

{'eval_loss': 1.305407166481018, 'eval_accuracy': 0.6127008327649361, 'eval_f1': 0.6185901881713457, 'eval_precision': 0.7111287250816293, 'eval_recall': 0.6127008327649361, 'eval_runtime': 1760.4106, 'eval_samples_per_second': 863.638, 'eval_steps_per_second': 107.955}


{'eval_loss': 1.5101377964019775, 'eval_accuracy': 0.6222867392329565, 'eval_f1': 0.6333954173319638, 'eval_precision': 0.7063302996226237, 'eval_recall': 0.6222867392329565, 'eval_runtime': 1727.4689, 'eval_samples_per_second': 880.107, 'eval_steps_per_second': 110.014}


{'eval_loss': 1.4635789394378662, 'eval_accuracy': 0.6310281072142925, 'eval_f1': 0.642322279622404, 'eval_precision': 0.7047278429070671, 'eval_recall': 0.6310281072142925, 'eval_runtime': 1777.2744, 'eval_samples_per_second': 855.443, 'eval_steps_per_second': 106.931}


{'eval_loss': 1.8006943464279175, 'eval_accuracy': 0.6245750175781083, 'eval_f1': 0.6338984615289801, 'eval_precision': 0.7003925736818132, 'eval_recall': 0.6245750175781083, 'eval_runtime': 1772.5512, 'eval_samples_per_second': 857.722, 'eval_steps_per_second': 107.216}


In [ ]:
print(results)

{'checkpoint_160000': {'eval_loss': 1.305407166481018, 'eval_accuracy': 0.6127008327649361, 'eval_f1': 0.6185901881713457, 'eval_precision': 0.7111287250816293, 'eval_recall': 0.6127008327649361, 'eval_runtime': 1760.4106, 'eval_samples_per_second': 863.638, 'eval_steps_per_second': 107.955}, 'checkpoint_200000': {'eval_loss': 1.5101377964019775, 'eval_accuracy': 0.6222867392329565, 'eval_f1': 0.6333954173319638, 'eval_precision': 0.7063302996226237, 'eval_recall': 0.6222867392329565, 'eval_runtime': 1727.4689, 'eval_samples_per_second': 880.107, 'eval_steps_per_second': 110.014}, 'checkpoint_220000': {'eval_loss': 1.4635789394378662, 'eval_accuracy': 0.6310281072142925, 'eval_f1': 0.642322279622404, 'eval_precision': 0.7047278429070671, 'eval_recall': 0.6310281072142925, 'eval_runtime': 1777.2744, 'eval_samples_per_second': 855.443, 'eval_steps_per_second': 106.931}, 'checkpoint_280000': {'eval_loss': 1.8006943464279175, 'eval_accuracy': 0.6245750175781083, 'eval_f1': 0.63389846152898